## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-24-02-31-00 +0000,wsj,Japan-U.S. Trade Deal Likely Positive for Yen ...,https://www.wsj.com/finance/currencies/asian-c...
1,2025-07-24-02-25-42 +0000,bbc,White House hits back at reports Trump named i...,https://www.bbc.com/news/articles/cwyq921zqqzo
2,2025-07-24-02-23-42 +0000,nyt,Appeals Court Blocks Trump’s Attempt to Restri...,https://www.nytimes.com/2025/07/23/us/politics...
3,2025-07-24-02-10-37 +0000,bbc,Columbia University to pay $200m in settlement...,https://www.bbc.com/news/articles/cq8zljpvyk0o
4,2025-07-24-02-05-27 +0000,nyt,Republicans Are Breaking With Trump Over Epste...,https://www.nytimes.com/2025/07/23/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2299/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
12,trump,57
125,new,20
2,deal,18
1,trade,15
470,he,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
18,2025-07-24-01-00-00 +0000,wsj,President Trump’s push to introduce a new stan...,https://www.wsj.com/economy/trade/trump-trade-...,145
305,2025-07-23-09-24-00 +0000,wsj,President Trump announced a trade deal with Ja...,https://www.wsj.com/economy/trade/u-s-japan-st...,126
108,2025-07-23-21-45-42 +0000,nypost,Third round of Ukraine-Russia peace talks end ...,https://nypost.com/2025/07/23/us-news/ukraine-...,123
195,2025-07-23-18-03-36 +0000,nyt,How Japan Clinched a Trade Deal With Trump Jus...,https://www.nytimes.com/2025/07/23/business/tr...,115
56,2025-07-23-23-26-00 +0000,wsj,The Trump administration issued a new AI actio...,https://www.wsj.com/tech/ai/trump-pledges-move...,113


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
18,145,2025-07-24-01-00-00 +0000,wsj,President Trump’s push to introduce a new stan...,https://www.wsj.com/economy/trade/trump-trade-...
108,66,2025-07-23-21-45-42 +0000,nypost,Third round of Ukraine-Russia peace talks end ...,https://nypost.com/2025/07/23/us-news/ukraine-...
210,64,2025-07-23-16-39-41 +0000,nyt,Bryan Kohberger Said He Wanted to Counsel High...,https://www.nytimes.com/2025/07/23/us/bryan-ko...
205,50,2025-07-23-16-59-32 +0000,nypost,"Emanuel, Brigitte Macron sue Candace Owens for...",https://nypost.com/2025/07/23/media/candace-ow...
317,48,2025-07-23-06-49-36 +0000,nypost,In-N-Out heiress Lynsi Snyder defends decision...,https://nypost.com/2025/07/23/business/in-n-ou...
61,47,2025-07-23-23-17-00 +0000,wsj,Tulsi Gabbard and the White House alleged form...,https://www.wsj.com/politics/policy/tulsi-gabb...
179,41,2025-07-23-19-08-00 +0000,wsj,Justice Department told Trump in May that his ...,https://www.wsj.com/politics/justice-departmen...
145,39,2025-07-23-20-30-00 +0000,wsj,Bryan Kohberger Given Four Life Sentences in K...,https://www.wsj.com/us-news/law/bryan-kohberge...
231,37,2025-07-23-15-15-23 +0000,nypost,Ex-DOGE lawyer launches AI policy council to p...,https://nypost.com/2025/07/23/us-news/ex-doge-...
194,34,2025-07-23-18-06-24 +0000,nyt,TV Show Helps Identify Mother and Child Found ...,https://www.nytimes.com/2025/07/22/world/europ...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
